In [1]:
import os
import pandas as pd

In [2]:
data_train = pd.read_json('train.json')
data_train

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative
...,...,...,...
8258,"Как мы писали еще весной, для увеличения сбыта...",10312,positive
8259,Но молодой министр национальной экономики Биши...,10313,negative
8260,\n \nВ ЕНПФ назначен новый председатель правле...,10314,neutral
8261,В Алматы у отделения банка произошло нападение...,10315,negative


In [3]:
data_test = pd.read_json('test.json')
data_test

,text,id
0,"Как сообщает пресс-служба акимата Алматы, для ...",0
1,Казахстанские авиакомпании перевозят 250 тысяч...,1
2,На состоявшемся под председательством Касым-Жо...,2
3,В ОАЭ состоялись переговоры между казахстанско...,3
4,12 вагонов грузового поезда сошли с путей в Во...,4
...,...,...
2051,На официальной странице общественного движения...,2079
2052,"официальный курс – 330,55 тенге за Доллар США ...",2083
2053,"«Базовая ставка, которая сейчас составляет 12%...",2084
2054,На начальном этапе за неоплату парковки на при...,2087


In [4]:
class_dict={
            'positive':1,
            'negative': 2,
            'neutral': 3
        }

# Mapping the classes
data_train['class'] = data_train['sentiment'].map(class_dict)
data_train.head()

,text,id,sentiment,class
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative,2
1,Медики рассказали о состоянии пострадавшего му...,1957,negative,2
2,"Прошел почти год, как железнодорожным оператор...",1969,negative,2
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative,2
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative,2


In [5]:
data_train['class'].value_counts()

3    4034
1    2795
2    1434
Name: class, dtype: int64

In [26]:
# Lets do some cleaning of this text
def clean_it(text,normalize=True):
    # Replacing possible issues with data. We can add or reduce the replacemtent in this chain
    s = str(text).replace(',',' ').replace('"','').replace('\'',' \' ').replace('.',' . ').replace('(',' ( ').\
            replace(')',' ) ').replace('!',' ! ').replace('?',' ? ').replace(':',' ').replace(';',' ').lower()

    # normalizing / encoding the text
    if normalize:
        s = s.normalize('NFKD').str.encode('ascii','ignore').str.decode('utf-8')

    return s

# Now lets define a small function where we can use above cleaning on datasets
def clean_df(data, cleanit= False, shuffleit=False, encodeit=False, label_prefix='__class__'):
    # Defining the new data
    df = data[['text', 'id']].copy(deep=True)
    df['class'] = label_prefix + data['class'].astype(str) + ' '

    # cleaning it
    if cleanit:
        df['text'] = df['text'].apply(lambda x: clean_it(x,encodeit))
        df['id'] = df['id'].apply(lambda x: clean_it(x,encodeit))

    # shuffling it
    if shuffleit:
        df.sample(frac=1).reset_index(drop=True)

    return df

In [27]:
%%time
# Transform the datasets using the above clean functions
data_train_cleaned = clean_df(data_train, True, True)

CPU times: user 505 ms, sys: 32.7 ms, total: 537 ms
Wall time: 546 ms


In [28]:
data_train_cleaned

,text,id,class
0,досудебное расследование по факту покупки енпф...,1945,__class__2
1,медики рассказали о состоянии пострадавшего му...,1957,__class__2
2,прошел почти год как железнодорожным оператор...,1969,__class__2
3,по итогам 12 месяцев 2016 года на территории р...,1973,__class__2
4,астана . 21 ноября . kazakhstan today - аген...,1975,__class__2
...,...,...,...
8258,как мы писали еще весной для увеличения сбыта...,10312,__class__1
8259,но молодой министр национальной экономики биши...,10313,__class__2
8260,\n \nв енпф назначен новый председатель правле...,10314,__class__3
8261,в алматы у отделения банка произошло нападение...,10315,__class__2


In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test= train_test_split(data_train_cleaned.drop(['class'], axis=1), data_train_cleaned['class'], test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6610, 2), (1653, 2), (6610,), (1653,))

In [31]:
train = pd.concat([y_train, X_train], axis = 1).reset_index(drop=True)
train

,class,text,id
0,__class__3,*в казахстане нет места для аэс\n* казахстанск...,6464
1,__class__2,по итогам трех кварталов 2016 года автоваз сни...,8774
2,__class__1,\n\nв алматы состоялся запуск нового котлоагре...,8807
3,__class__1,более 25 млн собрано для детей с нарушениями н...,5429
4,__class__3,ситуация на валютном рынке в настоящее время о...,5476
...,...,...,...
6605,__class__1,\n\nв рамках исполнения поручения главы госуда...,7788
6606,__class__3,глава мир рк женис касымбек предложил разрешит...,7245
6607,__class__1,пассажирам тулпар-тальго встречающим новогодн...,7444
6608,__class__3,на правительственном часе по налоговым и тамож...,2914


In [32]:
test = pd.concat([y_test, X_test], axis = 1).reset_index(drop=True)
test

,class,text,id
0,__class__3,в немецком городе гамбург состоялся экономичес...,7230
1,__class__3,глава правительства бакытжан сагинтаев провел ...,7472
2,__class__1,в среду 25 января первый заместитель п...,2910
3,__class__1,kapital . kz 19 января 2017 17 11 \n\nkapital...,10254
4,__class__3,высококачественные подделки тенге в казахстане...,3847
...,...,...,...
1648,__class__2,единый накопительный пенсионный фонд ( енпф )...,4115
1649,__class__1,одним из основных трендов кредитного рынка в 2...,4203
1650,__class__1,жилстройсбербанк зачислит первый транш компенс...,9188
1651,__class__3,астана . казинформ - до конца 2017 года в аст...,7975


In [37]:
# Write files to disk as fastText classifier API reads files from disk.
train_file =  'train.csv'
train.to_csv(train_file, header=None, index=False, columns=['class','text', 'id'] )
test_file =  'test.csv'
test.to_csv(test_file, header=None, index=False, columns=['class','text', 'id'] )

In [11]:
!pip install fastText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=2abbd290f8601ad2b6d3273f25e428e5a2228247d70b7143826056ab8a17c6fa
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fastText


In [38]:
from fasttext import train_supervised
"""fastText expects and training file (csv), a model name as input arguments.
label_prefix refers to the prefix before label string in the dataset.
default is __label__. In our dataset, it is __class__.
There are several other parameters which can be seen in:
https://pypi.org/project/fasttext/
"""
model = train_supervised(input=train_file, label="__class__", lr=1.0, epoch=75, loss='ova', wordNgrams=2, dim=200, thread=2, verbose=100)

In [39]:
for k in range(1,6):
    results = model.test(test_file,k=k)
    print(f"Test Samples: {results[0]} Precision@{k} : {results[1]*100:2.4f} Recall@{k} : {results[2]*100:2.4f}")

Test Samples: 1653 Precision@1 : 63.8234 Recall@1 : 63.8234
Test Samples: 1653 Precision@2 : 44.1319 Recall@2 : 88.2638
Test Samples: 1653 Precision@3 : 33.3333 Recall@3 : 100.0000
Test Samples: 1653 Precision@4 : 33.3333 Recall@4 : 100.0000
Test Samples: 1653 Precision@5 : 33.3333 Recall@5 : 100.0000


In [54]:
from tqdm.auto import tqdm

In [59]:
ress1 = []
for sentence in tqdm(data_test['text']):
    # print(sentence)
    res = {}
    sentence = sentence.replace("\n"," ")
    res['text'] = sentence
    # print(res['text'])
    preds = model.predict(sentence)[0]
    # print(preds)
    if preds[0] == '__class__1':
                  label = 'positive'
    elif preds[0] == '__class__2':
                  label = 'negative'
    else:
                  label = 'neutral'
    res['sentiment'] = label
    ress1.append(res)

  0%|          | 0/2056 [00:00<?, ?it/s]

In [60]:
itog1 = pd.DataFrame.from_dict(ress1)
itog1

,text,sentiment
0,"Как сообщает пресс-служба акимата Алматы, для ...",positive
1,Казахстанские авиакомпании перевозят 250 тысяч...,positive
2,На состоявшемся под председательством Касым-Жо...,neutral
3,В ОАЭ состоялись переговоры между казахстанско...,positive
4,12 вагонов грузового поезда сошли с путей в Во...,negative
...,...,...
2051,На официальной странице общественного движения...,neutral
2052,"официальный курс – 330,55 тенге за Доллар США ...",neutral
2053,"«Базовая ставка, которая сейчас составляет 12%...",positive
2054,На начальном этапе за неоплату парковки на при...,neutral


In [61]:
itog1.value_counts("sentiment")

sentiment
positive    960
neutral     845
negative    251
dtype: int64